In [ ]:
!pip install tensorflow==2.2.0

     |████████████████████████████████| 516.2MB 25kB/s 
     |████████████████████████████████| 3.0MB 41.9MB/s 
     |████████████████████████████████| 460kB 52.6MB/s 
  Found existing installation: tensorboard 2.4.1
    Uninstalling tensorboard-2.4.1:
      Successfully uninstalled tensorboard-2.4.1
  Found existing installation: tensorflow-estimator 2.4.0
    Uninstalling tensorflow-estimator-2.4.0:
      Successfully uninstalled tensorflow-estimator-2.4.0
  Found existing installation: tensorflow 2.4.1
    Uninstalling tensorflow-2.4.1:
      Successfully uninstalled tensorflow-2.4.1


In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Conv2D, BatchNormalization, Dropout, Flatten, Input, Dense, MaxPooling2D, Reshape
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model, model_from_json, load_model
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras import backend as K
from PIL import Image, ImageOps
import pickle
import time
import tqdm
import math

In [ ]:
tf.__version__

'2.2.0'

In [ ]:
from google.colab import drive 
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
X_train = []
Y_train = []
base_path = '/content/gdrive/MyDrive/data_for_training&test'
for i in tqdm.tqdm(range(1,4)):
  file_name = 'train_clselfie_v4_' + str(i) + ".pickle"
  pic = pickle.load(open(os.path.join(base_path, file_name), "rb"))
  for i in pic['X']:
    x = np.array(i)
    X_train.append(x)
  for i in pic['Y']:
    y = np.array(i)
    Y_train.append(y)


file_name = 'test_clselfie_v4.pickle'
X_val = []
Y_val = []
pic = pickle.load(open(os.path.join(base_path, file_name), "rb"))
for i in pic['X']:
  x = np.array(i)
  X_val.append(x)
for i in pic['Y']:
  y = np.array(i)
  Y_val.append(y)

100%|██████████| 3/3 [00:34<00:00, 11.59s/it]


In [ ]:
X_train = np.array(X_train)
Y_train = np.array(Y_train)
X_val = np.array(X_val)
Y_val = np.array(Y_val)
print(X_train.shape)

(28152, 208, 208, 1)


In [ ]:
input_cla = Input(shape=(208,208,1,))

fm_size = 16;
x = Conv2D(fm_size, (3,3), activation='relu', padding='same')(input_cla)
x = MaxPooling2D((2, 2), padding='same')(x)

fm_size = fm_size*2;
x = Conv2D(fm_size, (3,3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2), padding='same')(x)

fm_size = fm_size*2;
x = Conv2D(fm_size, (3,3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2), padding='same')(x)

fm_size = fm_size*2;
x = Conv2D(fm_size, (3,3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2), padding='same')(x)

fm_size = fm_size*2;
x = Conv2D(fm_size, (3,3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2), padding='same')(x)

fm_size = fm_size*2;
x = Conv2D(fm_size, (3,3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2), padding='same')(x)

fm_size = fm_size*2;
x = Conv2D(fm_size, (3,3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2), padding='same')(x)

fm_size = fm_size*2;
x = Conv2D(fm_size, (3,3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2), padding='same')(x)

em_size = math.ceil(208/(2**8));

x = Reshape((em_size*em_size*fm_size,),input_shape=(em_size,em_size,fm_size))(x)
x=Dense(50,activation='relu', input_shape=(em_size*em_size*fm_size,))(x)
x=Dense(50,activation='relu')(x)
x=Dense(10,activation='relu')(x)
output=Dense(5)(x)

classifier = Model(input_cla, output)
adam = Adam(lr=1e-4)
loss = MeanSquaredError()
classifier.compile(optimizer=adam, loss=loss, metrics = [tf.keras.metrics.MeanSquaredError()])

classifier.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 208, 208, 1)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 208, 208, 16)      160       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 104, 104, 16)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 104, 104, 32)      4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 52, 52, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 52, 52, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 26, 26, 64)        0     

In [ ]:
callbacks = [ReduceLROnPlateau(factor=0.3, patience=5, verbose = 1)]

In [ ]:
BATCH_SIZE = 64
TRAIN_STEP_SIZE = len(X_train) // BATCH_SIZE
history = classifier.fit(X_train, Y_train, batch_size = BATCH_SIZE, epochs = 50, steps_per_epoch = TRAIN_STEP_SIZE,
                    shuffle = True, validation_data = (X_val, Y_val))

Epoch 1/50
439/439 [==============================] - 40s 91ms/step - loss: 0.1331 - mean_squared_error: 0.1331 - val_loss: 0.0643 - val_mean_squared_error: 0.0643
Epoch 2/50
439/439 [==============================] - 41s 93ms/step - loss: 0.0396 - mean_squared_error: 0.0396 - val_loss: 0.0273 - val_mean_squared_error: 0.0273
Epoch 3/50
439/439 [==============================] - 40s 91ms/step - loss: 0.0228 - mean_squared_error: 0.0228 - val_loss: 0.0235 - val_mean_squared_error: 0.0235
Epoch 4/50
439/439 [==============================] - 40s 92ms/step - loss: 0.0191 - mean_squared_error: 0.0191 - val_loss: 0.0238 - val_mean_squared_error: 0.0238
Epoch 5/50
439/439 [==============================] - 40s 92ms/step - loss: 0.0164 - mean_squared_error: 0.0164 - val_loss: 0.0236 - val_mean_squared_error: 0.0236
Epoch 6/50
439/439 [==============================] - 40s 92ms/step - loss: 0.0139 - mean_squared_error: 0.0139 - val_loss: 0.0248 - val_mean_squared_error: 0.0248
Epoch 7/50
439/4

In [ ]:
model_json = classifier.to_json()
with open("/content/gdrive/MyDrive/ipv2.json", "w") as json_file:
    json_file.write(model_json)
classifier.save_weights("/content/gdrive/MyDrive/ipv2.h5")

In [ ]:
drive.flush_and_unmount()